# Temporal Node Embedding Property Approach

#### Import the required libraries
First of all we have to install and import the libraries that we need for the implementation of the Temporal Node Embedding.

- neo4j: The Neo4j Python driver is used to connect to the Neo4j database. 
- graphdatascience: The graph datascience client is a Python client for working with the Neo4j Graph Data Science Library which is used for the in-memory graph projection and the FastRP algorithm for the embedding.


In [ ]:
%pip install neo4j
%pip install graphdatascience

In [22]:
from neo4j import GraphDatabase
import graphdatascience

### Configure Driver and Client

We have to configure the driver and the client for the connection to the Neo4j database. The driver is used to execute Cypher queries and the client is used to execute the Graph Data Science Library algorithms. 

- Endpoint: Bolt URL of the Neo4j database 
- Username: Username 
- Password: Password
- database: Database where you imported the trips

In [97]:
gds = graphdatascience.GraphDataScience("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit"))
gds.set_database("neo4j")
db_driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "bachelorarbeit")).session(database="neo4j")

In [3]:
# Projection query
projection_query = """
MATCH (source)-[r:HAS_START|HAS_END]->(target)
WHERE source:Trip
WITH gds.graph.project(
  'propertyGraphWithProperties',
  source,
  target,
  {
  sourceNodeProperties: source { year: coalesce(source.validFrom.year, 0), month: coalesce(source.validFrom.month, 0), day: coalesce(source.validFrom.day, 0),
  hour: coalesce(source.validFrom.hour, 0), minute: coalesce(source.validFrom.minute, 0), weekday: coalesce(source.validFrom.dayOfWeek, 0)},
  targetNodeProperties: target { year: coalesce(target.validFrom.year, 0), month: coalesce(target.validFrom.month, 0), day: coalesce(target.validFrom.day, 0),
  hour: coalesce(target.validFrom.hour, 0), minute: coalesce(target.validFrom.minute, 0), weekday: coalesce(target.validFrom.dayOfWeek, 0)}
  },
  {undirectedRelationshipTypes: ['*']}
) as g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
"""

advanced_projection_query = """
MATCH (source)-[r:HAS_START|HAS_END]->(target)
WHERE source:Trip
WITH gds.graph.project(
  'propertyGraphWithProperties',
  source,
  target,
  {
    sourceNodeProperties: source { 
      year: coalesce(source.validFrom.year, 0), 
      month: coalesce(source.validFrom.month, 0), 
      day: coalesce(source.validFrom.day, 0),
      hour: coalesce(source.validFrom.hour, 0), 
      minute: coalesce(source.validFrom.minute, 0), 
      weekday: coalesce(source.validFrom.dayOfWeek, 0),
      season: CASE
                WHEN source.validFrom.month IN [12, 1, 2] THEN 1
                WHEN source.validFrom.month IN [3, 4, 5] THEN 2
                WHEN source.validFrom.month IN [6, 7, 8] THEN 3
                WHEN source.validFrom.month IN [9, 10, 11] THEN 4
                ELSE 0
              END,
      isWeekend: CASE
                   WHEN source.validFrom.dayOfWeek IN [6, 7] THEN 1
                   WHEN source.validFrom.dayOfWeek IN [1, 2, 3, 4, 5] THEN 0
                   ELSE 2
                 END
    },
    targetNodeProperties: target { 
      year: coalesce(target.validFrom.year, 0), 
      month: coalesce(target.validFrom.month, 0), 
      day: coalesce(target.validFrom.day, 0),
      hour: coalesce(target.validFrom.hour, 0), 
      minute: coalesce(target.validFrom.minute, 0), 
      weekday: coalesce(target.validFrom.dayOfWeek, 0),
      season: CASE
                WHEN target.validFrom.month IN [12, 1, 2] THEN 1
                WHEN target.validFrom.month IN [3, 4, 5] THEN 2
                WHEN target.validFrom.month IN [6, 7, 8] THEN 3
                WHEN target.validFrom.month IN [9, 10, 11] THEN 4
                ELSE 0
              END,
      isWeekend: CASE
                   WHEN target.validFrom.dayOfWeek IN [6, 7] THEN 1
                     WHEN target.validFrom.dayOfWeek IN [1, 2, 3, 4, 5] THEN 0
                   ELSE 2
                 END
    }
  },
  {undirectedRelationshipTypes: ['*']}
) AS g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
"""

run_query(advanced_projection_query)

[{'graph': 'propertyGraphWithProperties', 'nodes': 727085, 'rels': 2906174}]

In [89]:
def run_query(query, parameters=None):
    with db_driver as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

In [54]:
projection_query_id = """
MATCH (source)-[r:HAS_START|HAS_END]->(target)
WHERE source:Trip
WITH gds.graph.project(
  'propertyGraphWithPropertiesId',
  source,
  target,
  {
  sourceNodeProperties: source { year: coalesce(source.validFrom.year, source.stationId), month: coalesce(source.validFrom.month, source.stationId), day: coalesce(source.validFrom.day, source.stationId),
  hour: coalesce(source.validFrom.hour, source.stationId), minute: coalesce(source.validFrom.minute, source.stationId), weekday: coalesce(source.validFrom.dayOfWeek, source.stationId)},
  targetNodeProperties: target { year: coalesce(target.validFrom.year, target.stationId), month: coalesce(target.validFrom.month, target.stationId), day: coalesce(target.validFrom.day, target.stationId),
  hour: coalesce(target.validFrom.hour, target.stationId), minute: coalesce(target.validFrom.minute, target.stationId), weekday: coalesce(target.validFrom.dayOfWeek, target.stationId)}
  },
  {undirectedRelationshipTypes: ['*']}
) as g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
"""

run_query(projection_query_id)

[{'graph': 'propertyGraphWithPropertiesId', 'nodes': 727085, 'rels': 2906174}]

In [98]:
projection_query = """
MATCH (source)-[r:HAS_START|HAS_END]->(target)
WHERE source:Trip
WITH gds.graph.project(
  'propertyGraphWithPropertiesId',
  source,
  target,
  {
  sourceNodeProperties: source { 
    year: toFloat(coalesce(source.validFrom.year, source.stationId)), 
    month: toFloat(coalesce(source.validFrom.month, source.stationId)), 
    day: toFloat(coalesce(source.validFrom.day, source.stationId)),
    hour: toFloat(coalesce(source.validFrom.hour, source.stationId)), 
    minute: toFloat(coalesce(source.validFrom.minute, source.stationId)), 
    weekday: toFloat(coalesce(source.validFrom.weekday, source.stationId))
  },
  targetNodeProperties: target { 
    year: toFloat(coalesce(target.year, target.stationId)), 
    month: toFloat(coalesce(target.month, target.stationId)), 
    day: toFloat(coalesce(target.day, target.stationId)),
    hour: toFloat(coalesce(target.hour, target.stationId)), 
    minute: toFloat(coalesce(target.minute, target.stationId)), 
    weekday: toFloat(coalesce(target.weekday, target.stationId))
  }
  },
  {undirectedRelationshipTypes: ['*']}
) as g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
"""

run_query(projection_query_id)

[{'graph': 'propertyGraphWithPropertiesId', 'nodes': 727085, 'rels': 1453087}]

In [ ]:
G = gds.graph.get("propertyGraphWithPropertiesId")

result = gds.fastRP.write.estimate(
    G,
    writeProperty="temporalEmbeddingId",
    randomSeed = 42,
    embeddingDimension=128,
    propertyRatio = 0.5,
    featureProperties= ['day', 'hour', 'minute', 'weekday'],
    iterationWeights=[1.0]
)

result

In [ ]:
gds.fastRP.write(
    G,
    writeProperty="temporalEmbeddingId",
    randomSeed = 42,
    embeddingDimension=128,
    nodeSelfInfluence=0.8,
    propertyRatio = 0.5,
    featureProperties= ['day', 'hour', 'minute', 'weekday'],
    iterationWeights= [1.0]
)


In [92]:
G.drop()
gds.close()
db_driver.close()